# DRS4 baseline files 

In [ ]:
import tables
import matplotlib.pyplot as plt
import numpy as np
from lstcam_calib.io import get_dataset_keys


## HDF5 format

In [ ]:

file = "../../../test_data/real/service/PixelCalibration/Cat-A/drs4_baseline/20200218/pro/drs4_pedestal.Run02005.0000.h5"
hdf5_f=tables.open_file(file, "r")
hdf5_f

### Keys

In [ ]:
key=get_dataset_keys(file)
key 

### Data 

In [ ]:
# Date used for calibration are:
# - baseline_mean : baseline to be subtracted per channel/pixel/capacitor (2, 1855, 4096)
# - spike_height : spike to be subtracted, per channel/pixel/spike-type (2, 1855, 3)

drs4_data = hdf5_f.root.r1.monitoring.drs4_baseline.tel_001[0]
drs4_data["baseline_mean"]

In [ ]:
# other way, with ctapipe
from ctapipe.io import read_table
# read the table
table = read_table(file, key[0])
table

### Meta-data 

In [ ]:
hdf5_f.root._v_attrs 

In [ ]:
# plot baseline as function of capacitor for a given pixel
pixel=0
channel=["HG","LG"]
baseline=drs4_data["baseline_mean"]

fig, ax = plt.subplots(1, 2, figsize=(12,6),constrained_layout=True)
for chan in np.arange(2):
    label=f"channel {channel[chan]}"
    ax[chan].grid(True)
    ax[chan].set_ylim(380,420)
    ped_median=np.median(baseline[chan,pixel,:])
    ax[chan].plot(baseline[chan,pixel,:],label=label)
    ax[chan].axhline(ped_median, color='red',linestyle="--")
    ax[chan].set_ylabel(f'baseline [ADC]',fontsize=20) 
    ax[chan].set_xlabel(f'capacitor',fontsize=20) 
   
    ax[chan].legend(prop={'size':15})


In [ ]:
# plot the spikes
spike_data=drs4_data["spike_height"]
fig, ax = plt.subplots(1, 2, figsize=(12,6),constrained_layout=True)
for chan in np.arange(2):
    label=f"channel {channel[chan]}"
    ax[chan].set_title(label, fontsize=15)
    ax[chan].grid(True)
    
    for spike in np.arange(3):
        label=f"spike {spike}: median {np.median(spike_data[chan,:,spike]):3.1f}"
        ax[chan].hist(spike_data[chan,:,spike], label=label)
               
        ax[chan].set_xlabel(f'spikes [ADC]',fontsize=20) 
        ax[chan].set_ylabel(f'pixel',fontsize=20) 

        ax[chan].legend(prop={'size':15})


## Fits format 

In [ ]:
from astropy.io import fits

file = "../../../test_data/real/service/PixelCalibration/Cat-A/drs4_baseline/20200218/pro/drs4_pedestal.Run02005.0000.fits.gz"
fits_f = fits.open(file)


### HDU

In [ ]:
fits_f.info()

### Data

In [ ]:
fits_f["BASELINE_MEAN"].data

### Meta-data 

In [ ]:
fits_f["PRIMARY"].header

In [ ]:
fits_f["BASELINE_MEAN"].header